In [1]:
from time import perf_counter

from transformers import AutoTokenizer
import torch

import pandas as pd
from tqdm import tqdm

from models.prefix_tuning.gpt2 import GPT2PrefixTuningForCausalLM
from models.prefix_tuning.gptj import GPTJPrefixTuningForCausalLM

In [2]:
model_path = "/opt/ml/final-project-level3-nlp-08/outputs/checkpoint-459"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = GPT2PrefixTuningForCausalLM.from_pretrained(model_path).to(device)
model.eval()

GPT2PrefixTuningForCausalLM(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=

In [3]:
@torch.no_grad()
def genreate_from_input(input_text: str, max_length: int = 64, top_k: int = 30, top_p: float = 0.95):
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Check generation time
    t = perf_counter()

    output = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        do_sample=True,
        top_k=top_k,
        top_p=top_p,
        bad_words_ids=[[0], [2], [3], [4], [5], [6], [7], [8]],
    )
    generated_text = tokenizer.decode(output[0])

    spent = perf_counter() - t

    return generated_text, spent

In [5]:
prompt = "스키를 타고 눈 덮인 비탈을 내려오는 남자"
input_text = f"@{prompt}@<usr>\n"

for top_p in [0.95, 0.96, 0.97, 0.98, 0.99]:
    print(f"{top_p=} generated text")
    for n in range(1, 6):
        generated_text, spent = genreate_from_input(input_text, max_length=128, top_p=top_p)
        print(f"{n}:\n{generated_text}")
        # print(f"\ntime spent: {spent:.2f} sec")
    print()


top_p=0.95 generated text
1:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
아찔한 눈이
내 눈에 박혀있다
무릎을 펴고 몸을 웅크린 채,
주머니에
손을 넣었네
그리고 단돈 100원이 들어 있는
지갑을 찾아서
단숨에 주머니 속에 집어넣었다.
그 지갑이 사라졌다가 다시 보이면
또 다른 돈을 좇아야지
남자답다면 이런 일을 할 수 있을 것
눈은 여전히 날렵하다
그래서 남자들이 스키장을 즐겨 찾나보다 하고
말하려 하지만.
사라지는 눈을 바라보는 게 두려웠고
나는 그 자리에 섰다 그러나 나의 어깨에는 이미
2:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
그는, 나를 찾아온다. 스키장에 온 것은 그에게는 얼마나 영광인가. 나는 그를 보러온 것이다.  그의 입김에 내 몸을 맡긴다. 그는 나의 옷자락도 잡아당겨 보고 스르르 잠든다. 그러면
그런데 누구라도 그렇게 하지 않는가! 나라고 다르지 않건만.
오히려 나에게 더 가까이 다가와 있다. 아니 이제 내가 그에게로 가는 것 같다. 어느 날 아침이 되면 눈을 뜨는 그가 보인다. 아니, 이게 아니라 그냥 안일한 생각에서 일까!
난생처음 보는 눈이 시린 바람이다.
스윽슥,

3:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
난 왜 이렇게도 나를 싫어하는 걸까?

그런데도
슬픈 일이야. 그런 거.
너, 나에 관해 얼마나 알고 있을까.
나의 나쁜 버릇을 내 머릿속에 가지고 있다.
나란 존재가 밉고, 또 싫어지는 이유는 무엇일까, 나는 어떤 식으로든 알 수 있는걸까. 그렇지 않고서야 어떻게 그렇게 남을 미워하고 자신을 사랑하게 될 수가 있을까,
한심한 것들, 모두 내가 만든 업보겠지, 그래도 그것만은 안 되겠지.
나는 오늘로 몇 사람의 기억을 먹고 사는 사람이고 싶다.
이젠 나와 우리 사이에 놓인 기억의 고리를 끊고
4:
@스키를 타고 눈 덮인 비탈을 내려오는 남자@<usr> 
눈길.

누군가는 눈이 녹는다고 했겠지
결국 그 길은 겨울이면 차도를 건너뛰다 밟은 자취가 지워지는 길,